<a href="https://colab.research.google.com/github/CamilaKim/python_study/blob/master/ted_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install fake_useragent

In [0]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests

def extracturls(num,lang2):
  urllist=[]
  ua = UserAgent()
  header = {'User-Agent':str(ua.chrome)}
  res=requests.get('https://www.ted.com/talks?language='+lang2+'&page='+str(num)+'&sort=newest',headers=header).content
  soup=BeautifulSoup(res,'lxml')
  find_href=soup.find_all('a',class_='')
  for url in find_href:
    if url['href'].startswith("/talks"):
      urllist.append(url['href'][:-12])
  urllist2=list(set(urllist))
  return urllist2

def crawling(url,lang):
  subtitle=[]
  sub=[]
  ua = UserAgent()
  header = {'User-Agent':str(ua.chrome)}
  res=requests.get('https://www.ted.com'+url+'/transcript?language='+lang,headers=header).text
  soup=BeautifulSoup(res,'lxml').findAll('div',{'class':"Grid__cell flx-s:1 p-r:4"})
  for i in soup:
    raw=i.text.replace('\n\n\n', '\n\n')
    raw=raw.replace('\n',' ')
    raw=raw.strip()
    sub.extend(raw.split('\t'))
  for text in sub:
    if not text=='':
      subtitle.append(text+'\n')
  return subtitle

from google.colab import files
def to_save(lang,name,content):
  with open(lang+name+'.txt','w') as f:
    f.write(''.join(content))
#   files.download(lang+name+'.txt')
  return 1

def div(urllist,lang1,lang2):
  list1=[]
  list2=[]
  lan1='en'
  lan2='ko'
  for num in range(len(urllist)):
    if len(crawling(urllist[num],lang1))==len(crawling(urllist[num],lang2)):
      print(num,'success!!','https://www.ted.com'+urllist[num])
      list1.extend(crawling(urllist[num],lang1))
      list2.extend(crawling(urllist[num],lang2))
    else:
      print(num,'fail!!','https://www.ted.com'+urllist[num])
  return {lan1:list1,lan2:list2}

lang1='en'
lang2='ko'
urllist=[]
for num in range(0,85):
  urllist.extend(extracturls(num,lang2))
print(len(urllist))
match=div(urllist,lang1,lang2)
lan1=match[lang1]
lan2=match[lang2]
print(len(lan1),len(lan2))
to_save('enko','Ted',lan1)
to_save('koen','Ted',lan2)
  